# CAPSTONE PROJECT PART-1

## Import librarys

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

## Get the html code of the link

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html_page= r.text
soup = BeautifulSoup(html_page, "lxml")

## Extracting the table's values in 3 lists
to create a table is used  "table" and inside it are "tr" for each row and "td" for the elements inside of each cell, so first we need extract each row and the extract each element

In [3]:
h=[i for i in soup.find_all('tr')] 
del(h[-5:]) # elements that we don't need
potc=[] # for postalcode
boro=[] # for borough
neigh=[] # for neighbourhood
for j in range(1,290):
    potc.append(h[j].find_all('td')[0].text) # text is for extract just the text, instead of <td>text</td>
    boro.append(h[j].find_all('td')[1].text)
    neigh.append(h[j].find_all('td')[2].text)
neigh=[i.split('\n')[0] for i in neigh]

## Create the Dataframe
### 1) Drop all rows that we won't need

In [4]:
dic = {'Postalcode':potc,'Borough':boro,'Neighbourhood':neigh}
df = pd.DataFrame(dic)[['Postalcode','Borough','Neighbourhood']]

drops = df[df['Borough'] == 'Not assigned'].index # we don't need this rows
df = df.drop(labels=drops,axis = 0).reset_index(drop=True)


not_neigh = df[df['Neighbourhood'] == 'Not assigned'].index # not_neigh just have 1 element, so is not neccesary a for loop
df.iloc[not_neigh[0],2] = df.iloc[not_neigh[0],1] # replace Not assigned for the neighbourhood
df.shape

(212, 3)

### 2) Just unique values
When postalcode is repited we will join each neighbourhood's value with ',' and drop the rest

In [5]:
uni=pd.unique(df['Postalcode']) # the uniques postalcodes
for code in uni:
    ff=df[df['Postalcode'] == code].index # find the index of each postalcode
    if len(ff)>1: # if there are more than 1 index for each postalcode, we'll join each neighbourhood
        for ind in range(1,len(ff)):
            df.iloc[ff[0],2]=df.iloc[ff[0],2]+','+df.iloc[ff[ind],2]
for code in uni:  # just keep 1 value for each postal code and drop the rest 
    ff=df[df['Postalcode'] == code].index
    df=df.drop(ff[1:],axis=0)

df=df.reset_index(drop=True)
df =  df.sort_values(by=['Postalcode'])
df.shape

(103, 3)